<a href="https://colab.research.google.com/github/sirbez/GitRepo/blob/master/Code/notebook/gpu_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import math
import os
from random import shuffle
import constants as CONST
import cv2
import pickle

def get_size_statistics():
    heights = []
    widths = []
    img_count = 0
    DIR = CONST.TRAIN_DIR
    for img in os.listdir(CONST.TRAIN_DIR):
        path = os.path.join(DIR, img)
        data = cv2.imread(path)
        #data = np.array(Image.open(path))
        heights.append(data.shape[0])
        widths.append(data.shape[1])
        img_count += 1
    avg_height = sum(heights) / len(heights)
    avg_width = sum(widths) / len(widths)
    print("Average Height: " + str(avg_height))
    print("Max Height: " + str(max(heights)))
    print("Min Height: " + str(min(heights)))
    print('\n')
    print("Average Width: " + str(avg_width))
    print("Max Width: " + str(max(widths)))
    print("Min Width: " + str(min(widths)))

#get_size_statistics()


def label_img(name):
    word_label = name.split('.')[0]
    label = CONST.LABEL_MAP[word_label]
    label_arr = np.zeros(2)
    label_arr[label] = 1
    return label_arr


def prep_and_load_data():
    DIR = CONST.TRAIN_DIR
    data = []
    image_paths = os.listdir(DIR)
    shuffle(image_paths)
    count = 0
    for img_path in image_paths:
        label = label_img(img_path)
        path = os.path.join(DIR, img_path)
        image = cv2.imread(path)
        image = cv2.resize(image, (CONST.IMG_SIZE, CONST.IMG_SIZE))
        image = image.astype('float') / 255.0
        #image = image.reshape(-1, CONST.IMG_SIZE, CONST.IMG_SIZE, 3)
        data.append([image, label])
        count += 1
        print(count)
        if count == CONST.DATA_SIZE:
            break

    shuffle(data)

    #with open('train_data.pickle', 'wb') as train_d_file:
    #    pickle.dump(train_data, train_d_file)
    print(len(data))
    print('done')

    return data


ModuleNotFoundError: No module named 'constants'

In [6]:
'''
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers. normalization import BatchNormalization
import numpy as np
from keras.callbacks import TensorBoard
'''


import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

In [7]:
import os


In [8]:
def get_model():
    model = tf.keras.Sequential()#Sequential()
    model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(CONST.IMG_SIZE, CONST.IMG_SIZE, 3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())

    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())


    model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())

    model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2)) ##

    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3)) ##
    model.add(Dense(2, activation = 'softmax'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
    print('model prepared...')
    return model


import time
NAME = f'32x64x96x96_d2x64_d2_F256_d2x128_d3x2_{int(time.time())}.h5'
NAME_B = f'32x64x96x96_d2x64_d2_F256_d2x128_d3x2_{int(time.time())}'

tboard_log_dir = os.path.join("logs", NAME_B)
tensorboard = TensorBoard(log_dir=tboard_log_dir)
#model = load_model('my_model.h5')

In [9]:
CONST.DATA_SIZE = 18_000
CONST.IMG_SIZE = 110
CONST.SPLIT_RATIO = 0.8
data = np.array(prep_and_load_data())
train_size = int(CONST.DATA_SIZE * CONST.SPLIT_RATIO)
print('dats', len(data), train_size)

train_data = data[:train_size]
train_images = np.array([i[0] for i in train_data]).reshape(-1, CONST.IMG_SIZE, CONST.IMG_SIZE, 3)
train_labels = np.array([i[1] for i in train_data])
print('train data fetched..')

test_data = data[train_size:]
test_images = np.array([i[0] for i in test_data]).reshape(-1, CONST.IMG_SIZE, CONST.IMG_SIZE, 3)
test_labels = np.array([i[1] for i in test_data])


print('data fetched..')


NameError: name 'CONST' is not defined

In [ ]:
model = get_model()
model.save(NAME)

In [ ]:
history = model.fit(train_images, train_labels, batch_size = 50, epochs = 15, verbose = 1, validation_data=(test_images, test_labels), callbacks=[tensorboard])

In [ ]:
model.save('18_000.h5')


In [ ]:
import pickle

In [ ]:
with open('18_000_history.pickle', 'wb') as file:
    pickle.dump(history.history, file)

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter("./prediction.mp4", fourcc, 1.0, (400,400))

def write_video(out, frame):
    frame = cv2.resize(frame, (400, 400))
    out.write(frame)


In [ ]:
CONST.TEST_DIR = './test1'
import copy
def process_image(directory, img_path):
    path = os.path.join(directory, img_path)
    image = cv2.imread(path)
    image_copy = copy.deepcopy(image)
    print(image.shape)


    image = cv2.resize(image, (CONST.IMG_SIZE, CONST.IMG_SIZE))
    image_std = image.astype('float') / 255.0
    return image_copy, image_std

def get_testing_images():
    test_data = []
    image_test_data = []

    DIR = CONST.TEST_DIR
    image_paths = os.listdir(DIR)
    image_paths = image_paths[:200]
    count = 0
    for img_path in image_paths:
        image, image_std = process_image(DIR, img_path)

        image_std = image_std.reshape(-1, CONST.IMG_SIZE, CONST.IMG_SIZE, 3)
        pred = model.predict([image_std])
        arg_max = np.argmax(pred, axis=1)
        max_val = np.max(pred, axis=1)
        s = val_map[arg_max[0]] + ' - ' + str(max_val[0]*100) + '%'
        cv2.putText(image, s,
            bottomLeftCornerOfText,
            font,
            fontScale,
            fontColor,
            lineType)

        #cv2.imshow('img', image)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        write_video(out, image)


        test_data.append(image_std)
        image_test_data.append(image)
        count += 1
        print(count)
    out.release()

    return np.array(test_data), image_test_data

testing_images, image_test_data = get_testing_images()
testing_images_scale = np.array([i for i in testing_images]).reshape(-1, CONST.IMG_SIZE, CONST.IMG_SIZE, 3)

In [ ]:
predictions = model.predict(testing_images_scale)
arg_max = np.argmax(predictions, axis=1)
max_val = np.max(predictions, axis=1)

In [ ]:
val_map = {1: 'Dog', 0: 'Cat'}

font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (20,20)
fontScale              = 0.5
fontColor              = (255,255,255)
lineType               = 2


In [ ]:
for arg, val, img in zip(arg_max, max_val, image_test_data):
    img = img*255
    print(arg)
    s = val_map[arg] + ' - ' + str(val*100) + '%'
    cv2.putText(img, s,
        bottomLeftCornerOfText,
        font,
        fontScale,
        fontColor,
        lineType)
    cv2.imshow('img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()